In [1]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


/Users/infinity0106/Documents/Personal/jda_challenge


In [2]:
from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout, TimeDistributed, Lambda
from keras.callbacks import ModelCheckpoint
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from joblib import dump, load
import numpy as np




Using TensorFlow backend.


In [3]:
class JDAKeras():
    def __init__(self):
        self.regressor = Sequential()

    def add_layers(self, input, output):
        # Initialising the RNN
        self.regressor.add(LSTM(units=50,
                                activation='tanh',
                                input_shape=input,
                                return_sequences=True))
        self.regressor.add(Dropout(0.6))

        self.regressor.add(LSTM(units=50,
                                activation='tanh'))
        self.regressor.add(Dropout(0.6))

        # Adding the output layer
        # self.regressor.add(Lambda(lambda x: x[:, -output:, :])) # less output size
        self.regressor.add(Dense(1))  # same size output

        print(self.regressor.summary())

    def optimizers(self):
        # Compiling the RNN
        self.regressor.compile(
            optimizer="adam", loss='mean_squared_error', metrics=["accuracy"])

    def train(self, X_train, y_train, validation_data, epochs=100, batch_size=5000, currency="EURUSD1"):
        # Check points
        filepath = currency + \
            "-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
        checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                                     save_best_only=True, save_weights_only=False, period=1, mode='max')
        callbacks_list = [checkpoint]
        # Fitting the RNN to the Training set
        self.regressor.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                           callbacks=callbacks_list, validation_data=validation_data)

    def save_model(self, name):
        self.regressor.save(f"{name}.h5")

    def load_model(self, path):
        self.regressor = load_model(path)

    def predict(self, X_test):
        return self.regressor.predict(X_test)

In [4]:
if __name__ == "__main__":
    

SyntaxError: unexpected EOF while parsing (<ipython-input-4-be57ae97db13>, line 2)

In [ ]:
    df = pd.read_pickle("./preprocesed.pkl")
    X = df.drop(["sa_quantity"], axis=1)
    Y = df["sa_quantity"]

In [4]:
    x_train, x_test, y_train, y_test = train_test_split(
        X, Y, test_size=0.3, random_state=69)

In [5]:
    model = SGDRegressor(max_iter=100, tol=None, verbose=1, 
                         validation_fraction=0.3, early_stopping=True,
                        alpha=0.001)

In [6]:
    model.fit(x_train, y_train)

-- Epoch 1
Norm: 7.92, NNZs: 513, Bias: -0.004697, T: 4954194, Avg. loss: 0.247182
Total training time: 54.47 seconds.
-- Epoch 2
Norm: 8.02, NNZs: 513, Bias: 0.007797, T: 9908388, Avg. loss: 0.237949
Total training time: 268.38 seconds.
-- Epoch 3
Norm: 7.99, NNZs: 513, Bias: 0.002133, T: 14862582, Avg. loss: 0.237700
Total training time: 446.26 seconds.
-- Epoch 4
Norm: 8.00, NNZs: 513, Bias: -0.003424, T: 19816776, Avg. loss: 0.237749
Total training time: 632.34 seconds.
-- Epoch 5
Norm: 7.99, NNZs: 513, Bias: 0.007943, T: 24770970, Avg. loss: 0.237700
Total training time: 842.57 seconds.
-- Epoch 6
Norm: 8.00, NNZs: 513, Bias: -0.002423, T: 29725164, Avg. loss: 0.237729
Total training time: 1029.45 seconds.
-- Epoch 7
Norm: 7.99, NNZs: 513, Bias: -0.001590, T: 34679358, Avg. loss: 0.237695
Total training time: 1222.76 seconds.
-- Epoch 8
Norm: 7.99, NNZs: 513, Bias: 0.002376, T: 39633552, Avg. loss: 0.237725
Total training time: 1414.84 seconds.
-- Epoch 9
Norm: 7.99, NNZs: 513, Bi

SGDRegressor(alpha=0.001, average=False, early_stopping=True, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=100,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=None, validation_fraction=0.3, verbose=1,
             warm_start=False)

In [7]:
    y_pred = model.predict(x_test)

In [8]:
    print(mean_squared_error(y_test, y_pred))

0.6819911521688934


In [10]:
     dump(model, 'model_sklearn.joblib') 

['model_sklearn.joblib']

In [5]:
    tmp_model = load('model_sklearn.joblib')

In [12]:
    y_jupy = model.predict(x_test)
    y_load = tmp_model.predict(x_test)

In [13]:
    print(mean_squared_error(y_jupy, y_load))

0.0


In [ ]:
    print(x_test.shape)

In [16]:
    df = pd.read_pickle("./preprocesed_pred.pkl")
    X = df.drop(["sa_quantity","location","product","date",
                 "temp_max","temp_min","event"], axis=1)
    print(X.shape)
    Y = df["sa_quantity"]
    y_pred = tmp_model.predict(X)

(1697400, 513)


In [17]:
    df = pd.read_csv("./server/input_data_pred.csv")
    y_pred = np.absolute(np.round((y_pred * 13.035948087840143) + 8.604585583447076))
    df["sa_quantity"]=y_pred
    print(df["sa_quantity"].head())
    np.max(y_pred)


0     7.0
1    12.0
2     9.0
3     7.0
4    20.0
Name: sa_quantity, dtype: float64


45.0

In [18]:
    out_df = df[["location","product","date","temp_mean",
                 "temp_max","temp_min","sunshine_quant","event","price","sa_quantity"]]
    out_df.to_csv("./server/output_data_pred.csv",index=False)

In [13]:
    model = JDAKeras()
    model.load_model('./server/model_jda.h5')
    

In [14]:
    type(X)

pandas.core.frame.DataFrame

In [15]:
    X=X.to_numpy()
    X=X[:,:,np.newaxis]
    y_pred = model.predict(X)
    print(y_pred.shape)

KeyboardInterrupt: 

In [ ]:
    y_pred = y_pred.squeeze()
    y_pred_norm = np.round((y_pred * 0.9999999999999989) + 2.5899186438952162e-18)
    